In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def densenet(n_classes, filters=32):
    
    def bn_rl_conv(x, filters, kernel=1, strides=1):

        x = tf.keras.layers.BatchNormalization()(x)        
        x = tf.keras.layers.Activation(keras.activations.relu)(x)
        x = tf.keras.layers.Conv2D(filters, kernel, strides=strides, padding='same')(x)
        return x

    def dense_block(x, repetition):  # 줄기를 기준으로 새로운 가지 형성, 새로운 가지만들고 array에 새로운 가지 만들때마다 넣어줌
        array = [x]
        for _ in range(repetition):
            y = bn_rl_conv(x, 4 * filters)
            y = bn_rl_conv(y, filters, 3)
            array.append(y)
            x = tf.keras.layers.concatenate(array)
        return x

    def transition_layer(x):
        x = tf.keras.layers.Conv2D(12, (1, 1), strides=2, padding='same')(x)
        x = tf.keras.layers.AvgPool2D(2, strides=2, padding='same')(x)
        return x

    input = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.layers.Conv2D(64, 7, strides=2, padding='same')(input)
    x = tf.keras.layers.MaxPool2D(3, strides=2, padding='same')(x)

    for repetition in [6, 12, 24, 16]:
        d = dense_block(x, repetition)
        x = transition_layer(d)

    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    output = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(input, output)
    return model


model = densenet(10)
model.summary()